# Flip 7

In [19]:
import random
import gymnasium as gym
import numpy as np

## Game Classes

### Card

In [3]:
class Card:
    def __init__(self, val: str):
        self.val = int(val)
        self.label = val

    def __repr__(self) -> str:
        return self.label

### Deck

In [20]:
cardList = {
    "0": 1,
    "1": 1,
    "2": 2,
    "3": 3,
    "4": 4,
    "5": 5,
    "6": 6,
    "7": 7,
    "8": 8,
    "9": 9,
    "10": 10,
    "11": 11,
    "12": 12,
    # "+2": 1,
    # "+4": 1,
    # "+6": 1,
    # "+8": 1,
    # "+10": 1,
    # "x2": 1,
    # "Freeze": 3,
    # "Flip Three": 3,
    # "Second Chance": 3
}

In [8]:
class Deck:
    def __init__(self):
        self.cards = []

        for c in cardList:
            for _ in range(cardList[c]):
                self.cards.append(Card(c))

    def shuffle(self) -> None:
        random.shuffle(self.cards)

    def draw(self) -> Card:
        return self.cards.pop() #Last term in list is top card in deck


    def __repr__(self) -> str:
        deckString = ""

        for c in self.cards:
            deckString += c.label + " "
        return deckString

### Player

In [21]:
class Player:
    def __init__(self):
        self.points = 0
        self.active = True
        self.hand = []

In [10]:
d = Deck()
d.shuffle()

print(d)

6 3 5 6 12 12 2 10 10 9 9 11 11 10 5 9 8 10 10 9 8 0 10 11 12 8 3 10 11 7 11 1 12 12 7 12 11 6 6 9 10 2 5 7 6 5 12 8 9 4 8 4 8 8 10 11 9 7 12 7 9 7 6 12 4 8 7 5 11 12 10 11 12 9 11 3 4 11 12 


## Env

In [23]:
class Flip7(gym.Env):
    def __init__(self, players: int = 2):
        self.players = players

        self.agent = Player()
        self.opponents = []
        self.deck = Deck()
        self.trash = np.array([])

        self.observation_space = gym.spaces.Dict(
            {
                "hand": gym.spaces.Box(0,2, shape=(21,), dtype = int),
                "points": gym.spaces.Box(0,485, shape=(1,), dtype=int),
                "oppHands": gym.spaces.Box(0,2, shape=(21,players - 1), dtype = int),
                "oppPoints": gym.spaces.MultiBinary([21,players - 1]),
                "Trash": gym.spaces.Box(0,12, shape=(21,), dtype = int)
            }
        )

        self.action_space = gym.spaces.Discrete(2) # 0 -> Hit, 1 -> Stay
    
    def _get_obs(self):
        hand = np.zeros(21)

        for c in self.agent.hand:
            hand[cardList.keys().index(c.label)] += 1

        oppHands = np.array([])
        for op in self.opponents:
            opHand = np.zeros(21)
            for c in op.hand:
                opHand[cardList.keys().index(c.label)] += 1

            oppHands.append(opHand)

        oppPoints = np.array([o.points for o in self.opponents])

        trash = np.zeros(21)

        for c in self.trash:
            trash[cardList.keys().index(c.label)] += 1
        
        return {"hand": hand, "points": self.agent.points, "oppHands": oppHands, "oppPoints": oppPoints, "Trash": trash}

In [16]:
observation_space = gym.spaces.Box(0,3,shape = (len(cardList), ), dtype = int)
observation_space.sample()

array([2, 0, 0, 1, 1, 2, 3, 2, 1, 3, 2, 3, 2])

2
